# tiling


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from util import get_edge_tile_px, split_img




In [ ]:
# get_edge_tile_px() # can tune parameter here

In [ ]:
%matplotlib inline

In [ ]:
# read in tf train example
import util 
import cv2
import io
#import importlib
#importlib.reload(util)
import tensorflow as tf
# from tensorflow_serving.apis import input_pb2, I want to use it parse tfrecord but fail
#from tensorflow_serving.apis import input_pb2
#from object_detection.protos.input_reader_pb2 import TFRecordInputReader

#from object_detection.core.box_list_ops import prune_completely_outside_window
#clip_to_window, concatenate, non_max_suppression, scale, change_coordinate_frame
from object_detection.utils import label_map_util, np_box_list, np_box_list_ops, dataset_util

PATH_TO_LABELS = '/mnt/c/Users/insleker/Google Drive/workspace/riceCV/data/rice_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)
from object_detection.utils import visualization_utils as viz_utils

In [ ]:
#tf_example= tf.train.Example()
#raw_dataset  = tf.data.TFRecordDataset( str(util.tr_tfrecord_path) )
writer = tf.io.TFRecordWriter( str( util.tile_tr_tfrecord_path ) )
raw_dataset = tf.compat.v1.io.tf_record_iterator(path=str(util.tr_tfrecord_path) ) #deprecated

T_ROW=5
T_COL=8
OVLP_PCT=1/4
# f=0
max_sample_num=0 #detect max sample num for config file
for e in raw_dataset:
    
    # 1. parsing each raw data to get a handlable data
    example = tf.train.Example()
    example.ParseFromString(e)
    height,width = (example.features.feature[i].int64_list.value[0] for i in ['image/height','image/width'] )
    filename, encoded_image_data, image_format=\
    (example.features.feature[i].bytes_list.value[0] for i in ['image/source_id','image/encoded','image/format'] )
    #y_min, x_min, y_max, x_max
    xmins, xmaxs, ymins, ymaxs = \
    (example.features.feature[i].float_list.value for i in ['image/object/bbox/xmin',
                                                            'image/object/bbox/xmax',
                                                            'image/object/bbox/ymin',
                                                            'image/object/bbox/ymax'] )
    classes_text= example.features.feature['image/object/class/text'].bytes_list.value
    classes= example.features.feature['image/object/class/label'].int64_list.value
    
    print( filename )
    boxes= np.stack( (ymins,xmins,ymaxs,xmaxs), axis=1 )
    boxlist=np_box_list.BoxList(boxes)
    print('ori boxes shape:', boxlist.num_boxes())
    
    #print(xmins)
    #print(classes_text)
    #break
    
    #openCV
    #https://stackoverflow.com/questions/44324944/how-to-decode-jpg-image-from-memory/44325163
    #img=np.frombuffer(encoded_image_data, dtype=np.uint8)
    
    #PIL
    #https://stackoverflow.com/questions/42036890/how-to-decode-jpg-png-in-python/42037963
    img_stream=io.BytesIO(encoded_image_data)
    #im=Image.open(img_stream)
    #im.show()
    
    #matplotlib
    img=mpimg.imread(img_stream, format=image_format )#numpy array
    img_stream.close()
    plt.figure( figsize=(6, 4) )
    plt.imshow(img)
    plt.show()
    
    
    # 2. start to slice the image 
    
    sub_img_list,clipped_boxlist_list= split_img(img,boxlist,T_ROW,T_COL,OVLP_PCT)
    # slice row
    for i in range(T_ROW):
        #slice col
        for j in range(T_COL):
            sub_img=sub_img_list[i][j]
            clipped_boxlist=clipped_boxlist_list[i][j]
            
            sample_num=clipped_boxlist.num_boxes()
            max_sample_num=sample_num if sample_num>max_sample_num else max_sample_num
            
            #verify by image
            image_np_with_detections = sub_img.copy()
            viz_utils.visualize_boxes_and_labels_on_image_array(
                  image_np_with_detections,
                  clipped_boxlist.get(),
                  sample_num*[1],
                  scores=None,
                  category_index=category_index,
                  use_normalized_coordinates=True,
                  max_boxes_to_draw=300,)
            plt.figure( figsize=(3, 3) )
            plt.imshow(image_np_with_detections)
            plt.show()
            
            
            #3.save to new tfrecord, (maybe could transfer to function)
            filename_split= filename.decode().split('.')
            #https://stackoverflow.com/questions/39641596/saving-a-numpy-array-as-a-io-bytesio-with-jpg-format
            encoded_sub_img= io.BytesIO()
            pil_sub_img= Image.fromarray(sub_img)
            
            new_format_str= 'jpeg' if image_format.decode().lower()=='jpg' else image_format.decode()
            pil_sub_img.save( encoded_sub_img, format=new_format_str ) #image_format.decode().lower()
            tf_example = tf.train.Example(features=tf.train.Features(feature={
              'image/height': dataset_util.int64_feature(sub_img.shape[0]),
              'image/width': dataset_util.int64_feature(sub_img.shape[1]),
              'image/filename': dataset_util.bytes_feature(
                  '{}.{}.{}.{}'.format(filename_split[0],i,j,filename_split[1]).encode() ),
              'image/source_id': dataset_util.bytes_feature(filename),
              'image/encoded': dataset_util.bytes_feature(encoded_sub_img.getvalue()),
              'image/format': dataset_util.bytes_feature(image_format),
              'image/object/bbox/xmin': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[1]),
              'image/object/bbox/xmax': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[3]),
              'image/object/bbox/ymin': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[0]),
              'image/object/bbox/ymax': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[2]),
              'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
              'image/object/class/label': dataset_util.int64_list_feature(sample_num*[1]),
            }))
            writer.write(tf_example.SerializeToString())
            encoded_sub_img.close()
    
    
    #img=cv2.imdecode( np.frombuffer(encoded_image_data, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    #print(img.shape)
    #np_box_list_ops.clip_to_window
#     break
raw_dataset.close()
writer.close()
print('max_sample_num:',max_sample_num)
print('finish')

## eval
code is same as front block, just change data to eval data

In [ ]:
#raw_dataset  = tf.data.TFRecordDataset( str(util.eval_tfrecord_path) )

writer = tf.io.TFRecordWriter( str( util.tile_eval_tfrecord_path ) )
raw_dataset = tf.compat.v1.io.tf_record_iterator(path=str(util.eval_tfrecord_path) ) #deprecated

T_ROW=5
T_COL=8
# f=0
max_sample_num=0 #detect max sample num for config file
for e in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(e)
    height,width = (example.features.feature[i].int64_list.value[0] for i in ['image/height','image/width'] )
    filename, encoded_image_data, image_format=\
    (example.features.feature[i].bytes_list.value[0] for i in ['image/source_id','image/encoded','image/format'] )
    #y_min, x_min, y_max, x_max
    xmins, xmaxs, ymins, ymaxs = \
    (example.features.feature[i].float_list.value for i in ['image/object/bbox/xmin',
                                                            'image/object/bbox/xmax',
                                                            'image/object/bbox/ymin',
                                                            'image/object/bbox/ymax'] )
    classes_text= example.features.feature['image/object/class/text'].bytes_list.value
    classes= example.features.feature['image/object/class/label'].int64_list.value
    
    print( filename )
    boxes= np.stack( (ymins,xmins,ymaxs,xmaxs), axis=1 )
    boxlist=np_box_list.BoxList(boxes)
    print('ori boxes shape:', boxlist.num_boxes())

    img_stream=io.BytesIO(encoded_image_data)

    img=mpimg.imread(img_stream, format=image_format )#numpy array
    img_stream.close()
    plt.figure( figsize=(6, 4) )
    plt.imshow(img)
    plt.show()
    #print(type(img))
    h,w,c=img.shape
    # slice row
    for i in range(T_ROW):
        WITH_OVLP_N_ROW, N_ROW= get_edge_tile_px(S=h, T=T_ROW, OVLP_PCT=1/4)
        #slice col
        for j in range(T_COL):
            WITH_OVLP_N_COL, N_COL= get_edge_tile_px(S=w, T=T_COL, OVLP_PCT=1/4)
            b_y_min= WITH_OVLP_N_ROW*i
            b_y_max= b_y_min+N_ROW
            b_x_min= WITH_OVLP_N_COL*j
            b_x_max= b_x_min+N_COL
            sub_img=img[ b_y_min:b_y_max , b_x_min:b_x_max]
            #print(sub_img.shape)
            
            
            #chage box
            window= [b_y_min/h, b_x_min/w, b_y_max/h, b_x_max/w] #normalized
            clipped_boxlist = np_box_list_ops.clip_to_window( boxlist=boxlist , window=window)
            #print('new slice {} {} boxes shape:{}'.format(i,j, clipped_boxlist.data['boxes'].shape) )
            clipped_boxlist= np_box_list_ops.change_coordinate_frame( boxlist=clipped_boxlist, window=window)
            sample_num=clipped_boxlist.num_boxes()
            max_sample_num=sample_num if sample_num>max_sample_num else max_sample_num
            
            #verify by image
            image_np_with_detections = sub_img.copy()
            viz_utils.visualize_boxes_and_labels_on_image_array(
                  image_np_with_detections,
                  clipped_boxlist.get(),
                  sample_num*[1],
                  scores=None,
                  category_index=category_index,
                  use_normalized_coordinates=True,
                  max_boxes_to_draw=300,)
            plt.figure( figsize=(3, 3) )
            plt.imshow(image_np_with_detections)
            plt.show()
            
            
            #save into drive
            filename_split= filename.decode().split('.')
            #https://stackoverflow.com/questions/39641596/saving-a-numpy-array-as-a-io-bytesio-with-jpg-format
            encoded_sub_img= io.BytesIO()
            sub_img= Image.fromarray(sub_img)
            
            new_format_str= 'jpeg' if image_format.decode().lower()=='jpg' else image_format.decode()
            sub_img.save( encoded_sub_img, format=new_format_str ) #image_format.decode().lower()
            tf_example = tf.train.Example(features=tf.train.Features(feature={
              'image/height': dataset_util.int64_feature(N_ROW),
              'image/width': dataset_util.int64_feature(N_COL),
              'image/filename': dataset_util.bytes_feature(
                  '{}.{}.{}.{}'.format(filename_split[0],i,j,filename_split[1]).encode() ),
              'image/source_id': dataset_util.bytes_feature(filename),
              'image/encoded': dataset_util.bytes_feature(encoded_sub_img.getvalue()),
              'image/format': dataset_util.bytes_feature(image_format),
              'image/object/bbox/xmin': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[1]),
              'image/object/bbox/xmax': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[3]),
              'image/object/bbox/ymin': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[0]),
              'image/object/bbox/ymax': dataset_util.float_list_feature(clipped_boxlist.get_coordinates[2]),
              'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
              'image/object/class/label': dataset_util.int64_list_feature(sample_num*[1]),
            }))
            writer.write(tf_example.SerializeToString())
            encoded_sub_img.close()
            
raw_dataset.close()
writer.close()
print('max_sample_num:',max_sample_num)
print('finish')

In [ ]:
#from object_detection.core.box_list_ops import prune_completely_outside_window
#clip_to_window, concatenate, non_max_suppression, scale, change_coordinate_frame
from object_detection.utils import np_box_list_ops
np_box_list_ops.clip_to_window

# compute_precision_recall
from object_detection.utils import metrics 

# inference
inference 的部份，還是回歸 inference 的流程，也是正常的流程，就是拿到大圖才tiling，tiling inference each image 再combine